# Make the WOD plot

This notebook creates the classic world ocean database stacked histrogram. It uses parquet files of lon, lat and time downloaded with the `download_wod.py` script

In [1]:
import pandas as pd
import requests
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

### fetch data

Run the next cell if you want to download the processed dataset from my server

In [2]:
if not Path('wod_by_year.pqt').exists():
    df = pd.read_parquet("http://callumrollo.com/files/wod_by_year.pqt")
    df.to_parquet('wod_by_year.pqt', index=False)

### Read downloaded data files

In [3]:
combined_file = Path('wod_by_year.pqt')
if combined_file.exists():
    df = pd.read_parquet(combined_file)
else:
    inputs = Path("downloaded_files").glob("*.pqt")
    df = pd.DataFrame()
    for infile in inputs:
        df_sub = pd.read_parquest(infile)
        try:
            df_sub["time"] = pd.to_datetime(df_sub["time"])
        except ValueError:
            print(f"fail for {infile}")
            continue
        df = pd.concat([df, df_sub])
    parts = df.ds_name.str.split('_', expand=True)
    df['file_year'] = parts[2]
    df['source'] = parts[1]
    df = df.sort_values('file_year')
    df['year_from_datetime'] = df.time.dt.year
    df = df.dropna(subset='year_from_datetime')
    df['year_from_datetime'] = df['year_from_datetime'].astype(int)
    df['country'] = df['country'].astype(str)
    df.to_parquet(combined_file, index=False)

Change the sources here if you want to switch up the order or exclude certain platforms

In [4]:
source_colors = {}
source_clean = {"osd": "Bottle",
                "mbt": "Mechanical BT",
                "xbt": "Expendable BT",
                "ctd": "CTD",
                "pfl": "Argo float",
                "drb": "Drifting buoy",
                "uor": "Undulating CTD",
                "gld": "Glider",
                "mrb": "Moored buoy",
                "apb": "Marine mammal",
}
for i, source in enumerate(source_clean.keys()):
    source_colors[f"{source}: {source_clean[source]}"] = f"C{i}"


In [ ]:
years_list = []
for source in source_clean.keys():
    df_sub = df[df.source==source]
    years_list.append(df_sub.year_from_datetime.values)
bins=np.arange(1900, 2025)

Use [matplotlib styles](https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html) to spice up the apperance 

In [ ]:
with plt.style.context('default'):
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.hist(years_list, bins, stacked=True, alpha=1, rwidth=0.8)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="center");
    ax.xaxis.set_major_locator(MultipleLocator(5))
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    ax.set(xlim=(1900, 2025), ylim=(0, 800), ylabel="number of vertical profiles per year (hundreds of thousands)")
    plt.yticks(ticks=np.arange(0, 900000, 100000), labels=np.arange(0, 9))
    plt.title("Changing methods of oceanographic data acquisition")
    ax.legend(source_colors)
    plt.savefig("wod-profiles.png", bbox_inches='tight')

### Additional filters

We can filter the data before making the plot, e.g. just keeping data from a subset of countries

In [ ]:
countries = df.country.unique()

In [ ]:
df_nations = df[df.country.isin(['SWEDEN',  'NORWAY','FINLAND',
       'GREAT BRITAIN', 'DENMARK', 'MONACO', 'GERMANY',
        'NETHERLANDS', 'BELGIUM', 'ITALY', 'AUSTRIA',
       'IRELAND', 'FRANCE', 'YUGOSLAVIA',
       'SPAIN',  'PORTUGAL', 'POLAND',
       'LITHUANIA', 'TURKEY', 'ICELAND',
        'GREECE',  'UKRAINE', 
       'EAST GERMANY',  'ROMANIA',
   'MALTA', 'CYPRUS',  'LATVIA', 'BULGARIA',
'ESTONIA',  'CROATIA', 'EUROPEAN UNION',])]

In [ ]:
years_list = []
for source in source_clean.keys():
    df_sub = df_nations[df_nations.source==source]
    years_list.append(df_sub.year_from_datetime.values)
bins=np.arange(1900, 2025)
fig, ax = plt.subplots(figsize=(10, 6))
plt.hist(years_list, bins, stacked=True, alpha=1, rwidth=0.8)
plt.setp(ax.get_xticklabels(), rotation=45, ha="center");
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.xaxis.set_minor_locator(MultipleLocator(1))
ax.set(xlim=(1900, 2025), ylim=(0, 100), ylabel="number of vertical profiles per year (tens of thousands)")
plt.yticks(ticks=np.arange(0, 110000, 10000), labels=np.arange(0, 11))
plt.title("Subset of oceanographic data from WOD")
ax.legend(source_colors)